In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from torchtext import data
from torchtext.data import TabularDataset
from tqdm.notebook import tqdm

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
chart = train[['text','target']]
t_chart = test[['text']]

In [4]:
def remove(text):
    text = text.str.lower()
    text = text.str.replace('http\S+','')# http://와 https:// 두 가지 버전 있음
    text = text.str.replace('[0-9]','')# 사람 숫자, 날짜
    text = text.str.replace('@\S+','')# '@' 뒤에는 아이디 태그
    text = text.str.replace(',','')
    return text

In [5]:
chart["text"] = remove(chart["text"])
t_chart["text"] = remove(t_chart["text"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
chart.to_csv('train_r.csv',index = False)
t_chart.to_csv('test_r.csv',index = False)

In [7]:
pd.read_csv('train_r.csv').target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(pd.read_csv('train_r.csv'))

7613

# CSV -> trainset (= TabularDataset)

In [9]:
BATCH_SIZE = 64
lr = 0.001
epochs = 20
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [10]:
TEXT = data.Field(sequential = True, tokenize = str.split, batch_first = True)
LABEL = data.LabelField(dtype = torch.long)

In [11]:
trainset = TabularDataset(path = 'train_r.csv', format='csv', fields=[('text', TEXT), ('label', LABEL)])
testset = TabularDataset(path = 'test_r.csv', format='csv', fields=[('text', TEXT)])

In [12]:
trainset.examples[0].text, trainset.examples[0].label

(['text'], 'target')

In [13]:
trainset.examples.pop(0) # column을 제거해야 합니다.

In [14]:
trainset.examples[0].text, trainset.examples[0].label

(['our',
  'deeds',
  'are',
  'the',
  'reason',
  'of',
  'this',
  '#earthquake',
  'may',
  'allah',
  'forgive',
  'us',
  'all'],
 '1')

# trainset -> train_iter (= torchtext.data.Iterator)

In [15]:
TEXT.build_vocab(trainset, min_freq = 2)# 최소 2번 이상 등장한 단어만 사전에 담음
LABEL.build_vocab(trainset)

In [16]:
trainset, valset = trainset.split(split_ratio = 0.9)

In [17]:
train_iter = data.Iterator(dataset = trainset, batch_size = BATCH_SIZE, device = DEVICE)
val_iter = data.Iterator(dataset = valset, batch_size = BATCH_SIZE, device = DEVICE)
test_iter = data.Iterator(dataset = testset, batch_size = BATCH_SIZE, device = DEVICE)

In [18]:
vocab_size = len(TEXT.vocab)
n_classes = 2

# model

In [19]:
class LSTM_model(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embedding_dim, n_classes):
        super(LSTM_model, self).__init__()
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embedding_dim)
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        # x = [64, 27]
        x = self.embed(x)
        # x = [64, 27, 128]
        h_0 = self._init_state(batch_size = x.size(0))#첫 번째 은닉 벡터 정의
        # h_0 = [1, 64, 256]
        x, _ = self.lstm(x,(h_0,h_0))
        # x = [64, 27, 256]
        h_t = x[:,-1,:]
        # h_t = [64, 256]
        logit = self.out(h_t)
        # logit = [64, 2]
        return logit
    
    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

# Train model

In [20]:
def train(model, optimizer, train_iter):
    model.train()
    acc, total_loss = 0, 0
    for b,batch in tqdm(enumerate(train_iter)):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        optimizer.zero_grad()# 기울기 0으로 초기화
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'mean')
        total_loss += loss.item()
        acc += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        loss.backward()
        optimizer.step()
    size = len(train_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100 * acc / size
    return avg_loss, avg_accuracy

# Evaluate model

In [21]:
def evaluate(model, val_iter):
    model.eval()
    acc, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'sum')#오차의 합 구하고 total_loss에 더해줌
        total_loss += loss.item()
        acc += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100 * acc / size
    return avg_loss, avg_accuracy

In [22]:
from tqdm import tqdm

# Train

In [23]:
model = LSTM_model(1, 128, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [29]:
model2 = LSTM_model(1, 256, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [30]:
model3 = LSTM_model(1, 512, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [26]:
from time import sleep

In [27]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  7.85it/s]
2it [00:00,  7.91it/s]
3it [00:00,  8.29it/s]
4it [00:00,  8.31it/s]
5it [00:00,  8.71it/s]
6it [00:00,  9.05it/s]
7it [00:00,  9.31it/s]
8it [00:00,  9.44it/s]
9it [00:00,  9.38it/s]
10it [00:01,  9.47it/s]
12it [00:01,  9.70it/s]
14it [00:01,  9.91it/s]
16it [00:01, 10.14it/s]
18it [00:01, 10.37it/s]
20it [00:02, 10.61it/s]
22it [00:02, 10.49it/s]
24it [00:02, 10.48it/s]
26it [00:02, 10.09it/s]
28it [00:02, 10.19it/s]
30it [00:03, 10.44it/s]
32it [00:03, 10.51it/s]
34it [00:03, 10.44it/s]
36it [00:03, 10.54it/s]
38it [00:03, 10.56it/s]
40it [00:03, 10.51it/s]
42it [00:04, 10.69it/s]
44it [00:04, 10.89it/s]
46it [00:04, 10.91it/s]
48it [00:04, 10.93it/s]
50it [00:04, 11.12it/s]
52it [00:05, 11.02it/s]
54it [00:05, 11.20it/s]
56it [00:05, 11.15it/s]
58it [00:05, 10.94it/s]
60it [00:05, 10.77it/s]
62it [00:05, 10.59it/s]
64it [

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.68>> <<val_accuracy : 58.00>>


  5%|████▏                                                                              | 1/20 [00:10<03:25, 10.80s/it]
0it [00:00, ?it/s]
2it [00:00, 10.90it/s]
3it [00:00, 10.41it/s]
5it [00:00, 10.93it/s]
7it [00:00, 10.94it/s]
9it [00:00, 10.98it/s]
10it [00:00, 10.46it/s]
11it [00:01, 10.31it/s]
12it [00:01,  9.93it/s]
13it [00:01,  9.80it/s]
15it [00:01,  9.98it/s]
16it [00:01,  9.83it/s]
17it [00:01,  9.82it/s]
18it [00:01,  9.78it/s]
19it [00:01,  9.55it/s]
21it [00:02, 10.01it/s]
23it [00:02, 10.23it/s]
25it [00:02, 10.31it/s]
27it [00:02, 10.13it/s]
29it [00:02, 10.05it/s]
31it [00:02, 10.24it/s]
33it [00:03, 10.11it/s]
35it [00:03, 10.16it/s]
37it [00:03, 10.12it/s]
39it [00:03,  9.91it/s]
40it [00:03,  9.93it/s]
41it [00:04,  9.91it/s]
43it [00:04, 10.17it/s]
45it [00:04, 10.53it/s]
47it [00:04, 10.50it/s]
49it [00:04, 10.37it/s]
51it [00:04, 10.56it/s]
53it [00:05, 10.09it/s]
55it [00:05,  9.86it/s]
56it [00:05,  9.90it/s]
57it [00:05,  9.92it/s]
58it [00:05,  9.62it/s]
60

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.67>> <<val_accuracy : 58.00>>


 10%|████████▎                                                                          | 2/20 [00:22<03:17, 10.97s/it]
0it [00:00, ?it/s]
1it [00:00,  9.23it/s]
2it [00:00,  9.11it/s]
3it [00:00,  8.49it/s]
4it [00:00,  8.44it/s]
5it [00:00,  7.86it/s]
6it [00:00,  8.01it/s]
8it [00:00,  8.32it/s]
9it [00:01,  7.81it/s]
10it [00:01,  8.13it/s]
11it [00:01,  8.58it/s]
13it [00:01,  9.16it/s]
14it [00:01,  9.36it/s]
16it [00:01,  9.86it/s]
17it [00:01,  9.64it/s]
19it [00:02, 10.12it/s]
21it [00:02, 10.42it/s]
23it [00:02, 10.63it/s]
25it [00:02, 10.76it/s]
27it [00:02, 10.73it/s]
29it [00:02, 11.01it/s]
31it [00:03, 11.39it/s]
33it [00:03, 11.55it/s]
35it [00:03, 11.73it/s]
37it [00:03, 12.07it/s]
39it [00:03, 12.05it/s]
41it [00:03, 11.85it/s]
43it [00:04, 11.67it/s]
45it [00:04, 11.71it/s]
47it [00:04, 11.41it/s]
49it [00:04, 11.27it/s]
51it [00:04, 11.14it/s]
53it [00:05, 11.14it/s]
55it [00:05, 10.98it/s]
57it [00:05, 10.94it/s]
59it [00:05, 11.11it/s]
61it [00:05, 11.04it/s]
63it 

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [04:02<00:29, 14.59s/it]
0it [00:00, ?it/s]
1it [00:00,  8.17it/s]
2it [00:00,  8.30it/s]
3it [00:00,  8.39it/s]
4it [00:00,  8.34it/s]
5it [00:00,  8.37it/s]
6it [00:00,  8.46it/s]
7it [00:00,  8.61it/s]
8it [00:00,  8.81it/s]
9it [00:01,  8.89it/s]
10it [00:01,  8.96it/s]
11it [00:01,  8.83it/s]
12it [00:01,  8.73it/s]
13it [00:01,  8.52it/s]
14it [00:01,  8.56it/s]
15it [00:01,  8.85it/s]
16it [00:01,  8.82it/s]
17it [00:01,  8.70it/s]
18it [00:02,  8.79it/s]
19it [00:02,  8.44it/s]
20it [00:02,  8.73it/s]
21it [00:02,  8.60it/s]
22it [00:02,  8.51it/s]
23it [00:02,  8.41it/s]
24it [00:02,  8.44it/s]
25it [00:02,  8.55it/s]
26it [00:03,  8.59it/s]
27it [00:03,  8.75it/s]
28it [00:03,  8.89it/s]
29it [00:03,  8.94it/s]
30it [00:03,  9.08it/s]
32it [00:03,  9.25it/s]
33it [00:03,  9.14it/s]
34it [00:03,  9.12it/s]
35it [00:03,  9.28it/s]
36it [00:04,  9.02it/s]
37it [00:04,  9.16it/s]
38it [

<<e : 19>> <<train_loss :  0.00>> <<train_accuracy : 98.00>> <<val_loss :  1.35>> <<val_accuracy : 72.00>>


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [04:14<00:13, 13.87s/it]
0it [00:00, ?it/s]
1it [00:00,  8.99it/s]
3it [00:00,  9.42it/s]
4it [00:00,  9.39it/s]
5it [00:00,  9.45it/s]
6it [00:00,  9.41it/s]
7it [00:00,  9.23it/s]
8it [00:00,  8.84it/s]
10it [00:01,  9.23it/s]
11it [00:01,  9.36it/s]
12it [00:01,  9.07it/s]
13it [00:01,  9.04it/s]
14it [00:01,  9.30it/s]
16it [00:01,  9.58it/s]
18it [00:01,  9.85it/s]
19it [00:01,  9.80it/s]
20it [00:02,  9.85it/s]
22it [00:02,  9.84it/s]
23it [00:02,  9.41it/s]
25it [00:02,  9.82it/s]
27it [00:02,  9.96it/s]
29it [00:02, 10.22it/s]
31it [00:03, 10.38it/s]
33it [00:03, 10.12it/s]
35it [00:03, 10.17it/s]
37it [00:03, 10.14it/s]
39it [00:03, 10.18it/s]
41it [00:04, 10.18it/s]
43it [00:04, 10.31it/s]
45it [00:04, 10.06it/s]
47it [00:04, 10.05it/s]
49it [00:04,  9.85it/s]
50it [00:05,  9.89it/s]
52it [00:05, 10.20it/s]
54it [00:05, 10.34it/s]
56it [00:05, 10.23it/s]
58it [00:05, 10.31it/s]
60it

<<e : 20>> <<train_loss :  0.00>> <<train_accuracy : 97.00>> <<val_loss :  1.28>> <<val_accuracy : 74.00>>


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:26<00:00, 13.32s/it]


In [31]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model2, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model2, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]


0it [00:00, ?it/s]


1it [00:00,  4.51it/s]


2it [00:00,  4.66it/s]


3it [00:00,  4.64it/s]


4it [00:00,  4.66it/s]


5it [00:01,  4.25it/s]


6it [00:01,  3.67it/s]


7it [00:01,  3.56it/s]


8it [00:02,  3.51it/s]


9it [00:02,  3.18it/s]


10it [00:02,  2.96it/s]


11it [00:03,  2.93it/s]


12it [00:03,  2.88it/s]


13it [00:03,  2.92it/s]


14it [00:04,  2.89it/s]


15it [00:04,  3.01it/s]


16it [00:04,  3.03it/s]


17it [00:05,  2.94it/s]


18it [00:05,  2.96it/s]


19it [00:05,  2.85it/s]


20it [00:06,  2.89it/s]


21it [00:06,  3.03it/s]


22it [00:06,  3.13it/s]


23it [00:07,  3.28it/s]


24it [00:07,  3.20it/s]


25it [00:07,  3.20it/s]


26it [00:08,  3.15it/s]


27it [00:08,  3.19it/s]


28it [00:08,  3.08it/s]


29it [00:09,  3.06it/s]


30it [00:09,  3.13it/s]


31it [00:09,  3.20it/s]


32it [00:10,  3.14it/s]


33it [00:10,  3.07it/s]


34it [

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 43.00>> <<val_loss :  0.70>> <<val_accuracy : 42.00>>




  5%|████▏                                                                              | 1/20 [00:30<09:31, 30.08s/it]


0it [00:00, ?it/s]


1it [00:00,  5.06it/s]


2it [00:00,  5.07it/s]


3it [00:00,  5.00it/s]


4it [00:00,  4.91it/s]


5it [00:01,  4.85it/s]


6it [00:01,  4.87it/s]


7it [00:01,  5.15it/s]


8it [00:01,  5.21it/s]


9it [00:01,  5.11it/s]


10it [00:01,  5.31it/s]


11it [00:02,  5.20it/s]


12it [00:02,  5.19it/s]


13it [00:02,  4.97it/s]


14it [00:02,  5.00it/s]


15it [00:02,  5.00it/s]


16it [00:03,  4.99it/s]


17it [00:03,  4.92it/s]


18it [00:03,  4.79it/s]


19it [00:03,  5.02it/s]


20it [00:03,  5.00it/s]


21it [00:04,  4.99it/s]


22it [00:04,  4.91it/s]


23it [00:04,  4.93it/s]


24it [00:04,  4.92it/s]


25it [00:05,  4.89it/s]


26it [00:05,  5.02it/s]


27it [00:05,  4.99it/s]


28it [00:05,  4.87it/s]


29it [00:05,  5.03it/s]


30it [00:06,  4.88it/s]


31it [00:06,  4.91it/s]


32it [00:06,  4.70it/s]


33it [00:06,  4.57it/s]


34it [

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 43.00>> <<val_loss :  0.70>> <<val_accuracy : 41.00>>




 10%|████████▎                                                                          | 2/20 [00:51<08:15, 27.55s/it]


0it [00:00, ?it/s]


1it [00:00,  5.09it/s]


2it [00:00,  5.08it/s]


3it [00:00,  4.89it/s]


4it [00:00,  4.78it/s]


5it [00:01,  4.79it/s]


6it [00:01,  4.89it/s]


7it [00:01,  4.92it/s]


8it [00:01,  5.21it/s]


9it [00:01,  5.33it/s]


10it [00:01,  5.33it/s]


11it [00:02,  5.51it/s]


12it [00:02,  5.46it/s]


13it [00:02,  5.29it/s]


14it [00:02,  5.21it/s]


15it [00:02,  5.19it/s]


16it [00:03,  5.17it/s]


17it [00:03,  5.11it/s]


18it [00:03,  5.10it/s]


19it [00:03,  5.08it/s]


20it [00:03,  5.02it/s]


21it [00:04,  5.10it/s]


22it [00:04,  5.31it/s]


23it [00:04,  5.35it/s]


24it [00:04,  5.27it/s]


25it [00:04,  5.25it/s]


26it [00:05,  5.12it/s]


27it [00:05,  5.09it/s]


28it [00:05,  5.05it/s]


29it [00:05,  5.04it/s]


30it [00:05,  5.06it/s]


31it [00:06,  4.78it/s]


32it [00:06,  4.72it/s]


33it [00:06,  4.75it/s]


34it [

...



 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [07:51<00:53, 26.94s/it]


0it [00:00, ?it/s]


1it [00:00,  4.41it/s]


2it [00:00,  4.63it/s]


3it [00:00,  4.58it/s]


4it [00:00,  4.61it/s]


5it [00:01,  4.60it/s]


6it [00:01,  4.53it/s]


7it [00:01,  4.64it/s]


8it [00:01,  4.53it/s]


9it [00:01,  4.54it/s]


10it [00:02,  4.63it/s]


11it [00:02,  4.76it/s]


12it [00:02,  4.79it/s]


13it [00:02,  4.83it/s]


14it [00:02,  4.95it/s]


15it [00:03,  5.03it/s]


16it [00:03,  4.93it/s]


17it [00:03,  5.10it/s]


18it [00:03,  4.82it/s]


19it [00:04,  4.66it/s]


20it [00:04,  4.73it/s]


21it [00:04,  4.72it/s]


22it [00:04,  4.76it/s]


23it [00:04,  4.82it/s]


24it [00:05,  4.93it/s]


25it [00:05,  4.69it/s]


26it [00:05,  4.70it/s]


27it [00:05,  4.71it/s]


28it [00:05,  4.78it/s]


29it [00:06,  4.78it/s]


30it [00:06,  4.76it/s]


31it [00:06,  4.79it/s]


32it [00:06,  4.63it/s]


33it [00:06,  4.51it/s]


34it [

<<e : 19>> <<train_loss :  0.01>> <<train_accuracy : 43.00>> <<val_loss :  0.70>> <<val_accuracy : 42.00>>




 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [08:14<00:25, 25.77s/it]


0it [00:00, ?it/s]


1it [00:00,  6.65it/s]


2it [00:00,  6.43it/s]


3it [00:00,  6.00it/s]


4it [00:00,  5.66it/s]


5it [00:00,  5.36it/s]


6it [00:01,  5.29it/s]


7it [00:01,  5.30it/s]


8it [00:01,  5.25it/s]


9it [00:01,  5.32it/s]


10it [00:01,  5.25it/s]


11it [00:02,  5.08it/s]


12it [00:02,  4.94it/s]


13it [00:02,  4.91it/s]


14it [00:02,  4.88it/s]


15it [00:02,  4.92it/s]


16it [00:03,  5.12it/s]


17it [00:03,  5.00it/s]


18it [00:03,  5.03it/s]


19it [00:03,  4.91it/s]


20it [00:03,  4.92it/s]


21it [00:04,  4.92it/s]


22it [00:04,  5.08it/s]


23it [00:04,  5.16it/s]


24it [00:04,  5.20it/s]


25it [00:04,  5.13it/s]


26it [00:05,  5.10it/s]


27it [00:05,  5.12it/s]


28it [00:05,  5.07it/s]


29it [00:05,  5.07it/s]


30it [00:05,  5.19it/s]


31it [00:06,  5.02it/s]


32it [00:06,  5.08it/s]


33it [00:06,  5.21it/s]


34it [

<<e : 20>> <<train_loss :  0.01>> <<train_accuracy : 43.00>> <<val_loss :  0.70>> <<val_accuracy : 42.00>>




100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [08:37<00:00, 25.87s/it]


In [32]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model3, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model3, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]


0it [00:00, ?it/s]


1it [00:00,  1.79it/s]


2it [00:01,  1.84it/s]


3it [00:01,  1.84it/s]


4it [00:02,  1.90it/s]


5it [00:02,  1.97it/s]


6it [00:03,  2.01it/s]


7it [00:03,  1.99it/s]


8it [00:04,  2.01it/s]


9it [00:04,  2.04it/s]


10it [00:05,  2.02it/s]


11it [00:05,  2.05it/s]


12it [00:06,  1.97it/s]


13it [00:06,  1.94it/s]


14it [00:07,  1.96it/s]


15it [00:07,  1.97it/s]


16it [00:08,  1.95it/s]


17it [00:08,  1.91it/s]


18it [00:09,  1.83it/s]


19it [00:09,  1.85it/s]


20it [00:10,  1.87it/s]


21it [00:10,  1.80it/s]


22it [00:11,  1.76it/s]


23it [00:11,  1.84it/s]


24it [00:12,  1.83it/s]


25it [00:13,  1.83it/s]


26it [00:13,  1.89it/s]


27it [00:14,  1.81it/s]


28it [00:14,  1.78it/s]


29it [00:15,  1.79it/s]


30it [00:15,  1.78it/s]


31it [00:16,  1.73it/s]


32it [00:17,  1.72it/s]


33it [00:17,  1.73it/s]


34it [

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 55.00>> <<val_loss :  0.69>> <<val_accuracy : 57.00>>




  5%|████▏                                                                              | 1/20 [01:03<19:57, 63.04s/it]


0it [00:00, ?it/s]


1it [00:00,  2.05it/s]


2it [00:00,  2.07it/s]


3it [00:01,  2.15it/s]


4it [00:01,  2.19it/s]


5it [00:02,  2.19it/s]


6it [00:02,  2.23it/s]


7it [00:03,  2.20it/s]


8it [00:03,  2.24it/s]


9it [00:04,  2.26it/s]


10it [00:04,  2.26it/s]


11it [00:04,  2.31it/s]


12it [00:05,  2.27it/s]


13it [00:05,  2.29it/s]


14it [00:06,  2.34it/s]


15it [00:06,  2.34it/s]


16it [00:06,  2.40it/s]


17it [00:07,  2.41it/s]


18it [00:07,  2.30it/s]


19it [00:08,  2.34it/s]


20it [00:08,  2.33it/s]


21it [00:09,  2.30it/s]


22it [00:09,  2.26it/s]


23it [00:10,  2.27it/s]


24it [00:10,  2.34it/s]


25it [00:10,  2.39it/s]


26it [00:11,  2.38it/s]


27it [00:11,  2.37it/s]


28it [00:12,  2.33it/s]


29it [00:12,  2.29it/s]


30it [00:13,  2.32it/s]


31it [00:13,  2.33it/s]


32it [00:13,  2.32it/s]


33it [00:14,  2.40it/s]


34it [

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.69>> <<val_accuracy : 57.00>>




 10%|████████▎                                                                          | 2/20 [01:53<17:46, 59.27s/it]


0it [00:00, ?it/s]


1it [00:00,  2.20it/s]


2it [00:01,  2.05it/s]


3it [00:01,  2.09it/s]


4it [00:02,  1.99it/s]


5it [00:02,  1.96it/s]


6it [00:03,  1.97it/s]


7it [00:03,  1.96it/s]


8it [00:04,  1.84it/s]


9it [00:04,  1.87it/s]


10it [00:05,  1.92it/s]


11it [00:05,  1.86it/s]


12it [00:06,  1.89it/s]


13it [00:06,  1.95it/s]


14it [00:07,  1.99it/s]


15it [00:07,  1.96it/s]


16it [00:08,  1.86it/s]


17it [00:08,  1.84it/s]


18it [00:09,  1.82it/s]


19it [00:10,  1.85it/s]


20it [00:10,  1.83it/s]


21it [00:11,  1.76it/s]


22it [00:11,  1.74it/s]


23it [00:12,  1.73it/s]


24it [00:12,  1.70it/s]


25it [00:13,  1.75it/s]


26it [00:14,  1.74it/s]


27it [00:14,  1.74it/s]


28it [00:15,  1.74it/s]


29it [00:15,  1.74it/s]


30it [00:16,  1.72it/s]


31it [00:17,  1.69it/s]


32it [00:17,  1.69it/s]


33it [00:18,  1.70it/s]


34it [

...



 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [17:16<01:55, 57.51s/it]


0it [00:00, ?it/s]


1it [00:00,  1.83it/s]


2it [00:01,  1.83it/s]


3it [00:01,  1.85it/s]


4it [00:02,  1.80it/s]


5it [00:02,  1.84it/s]


6it [00:03,  1.86it/s]


7it [00:03,  1.79it/s]


8it [00:04,  1.71it/s]


9it [00:05,  1.69it/s]


10it [00:05,  1.65it/s]


11it [00:06,  1.66it/s]


12it [00:06,  1.66it/s]


13it [00:07,  1.61it/s]


14it [00:08,  1.65it/s]


15it [00:08,  1.62it/s]


16it [00:09,  1.61it/s]


17it [00:10,  1.63it/s]


18it [00:10,  1.59it/s]


19it [00:11,  1.57it/s]


20it [00:11,  1.58it/s]


21it [00:12,  1.50it/s]


22it [00:13,  1.52it/s]


23it [00:14,  1.53it/s]


24it [00:14,  1.50it/s]


25it [00:15,  1.46it/s]


26it [00:16,  1.48it/s]


27it [00:16,  1.40it/s]


28it [00:17,  1.39it/s]


29it [00:18,  1.39it/s]


30it [00:19,  1.36it/s]


31it [00:19,  1.36it/s]


32it [00:20,  1.37it/s]


33it [00:21,  1.35it/s]


34it [

<<e : 19>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.69>> <<val_accuracy : 57.00>>




 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [18:18<00:58, 58.92s/it]


0it [00:00, ?it/s]


1it [00:00,  2.38it/s]


2it [00:00,  2.40it/s]


3it [00:01,  2.41it/s]


4it [00:01,  2.46it/s]


5it [00:02,  2.52it/s]


6it [00:02,  2.51it/s]


7it [00:02,  2.47it/s]


8it [00:03,  2.35it/s]


9it [00:03,  2.44it/s]


10it [00:04,  2.40it/s]


11it [00:04,  2.38it/s]


12it [00:04,  2.39it/s]


13it [00:05,  2.41it/s]


14it [00:05,  2.36it/s]


15it [00:06,  2.26it/s]


16it [00:06,  2.31it/s]


17it [00:07,  2.22it/s]


18it [00:07,  2.25it/s]


19it [00:08,  2.28it/s]


20it [00:08,  2.19it/s]


21it [00:08,  2.19it/s]


22it [00:09,  2.20it/s]


23it [00:09,  2.25it/s]


24it [00:10,  2.25it/s]


25it [00:10,  2.35it/s]


26it [00:11,  2.38it/s]


27it [00:11,  2.45it/s]


28it [00:11,  2.36it/s]


29it [00:12,  2.38it/s]


30it [00:12,  2.27it/s]


31it [00:13,  2.24it/s]


32it [00:13,  2.33it/s]


33it [00:14,  2.25it/s]


34it [

<<e : 20>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.69>> <<val_accuracy : 57.00>>




100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [19:11<00:00, 57.60s/it]
